# **DATA**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split

np.random.seed(42)

In [ ]:
label_mapping = {
    # DDoS
    'DDoS-ACK_Fragmentation': 'DDoS',
    'DDoS-UDP_Flood': 'DDoS',
    'DDoS-SlowLoris': 'DDoS',
    'DDoS-ICMP_Flood': 'DDoS',
    'DDoS-RSTFINFlood': 'DDoS',
    'DDoS-PSHACK_Flood': 'DDoS',
    'DDoS-HTTP_Flood': 'DDoS',
    'DDoS-UDP_Fragmentation': 'DDoS',
    'DDoS-TCP_Flood': 'DDoS',
    'DDoS-SYN_Flood': 'DDoS',
    'DDoS-SynonymousIP_Flood': 'DDoS',
    'DDoS-ICMP_Fragmentation': 'DDoS',
    
    # DoS
    'DoS-TCP_Flood': 'DoS',
    'DoS-HTTP_Flood': 'DoS',
    'DoS-SYN_Flood': 'DoS',
    'DoS-UDP_Flood': 'DoS',
    
    # Brute Force
    'DictionaryBruteForce': 'Brute Force',
    
    # Spoofing
    'MITM-ArpSpoofing': 'Spoofing',
    'DNS_Spoofing': 'Spoofing',
    
    # Recon
    'Recon-PingSweep': 'Recon',
    'Recon-OSScan': 'Recon',
    'VulnerabilityScan': 'Recon',
    'Recon-PortScan': 'Recon',
    'Recon-HostDiscovery': 'Recon',
    
    # Web-based
    'SqlInjection': 'Web-based',
    'CommandInjection': 'Web-based',
    'Backdoor_Malware': 'Web-based',
    'Uploading_Attack': 'Web-based',
    'XSS': 'Web-based',
    'BrowserHijacking': 'Web-based',
    
    # Mirai
    'Mirai-greip_flood': 'Mirai',
    'Mirai-greeth_flood': 'Mirai',
    'Mirai-udpplain': 'Mirai',
    
    # Benign Traffic
    'BenignTraffic': 'Benign'
}

def GroupAttacks(label):
    return label_mapping.get(label, 'Unknown') # Default to 'unknown'

def preprocessing(df):
    """Initial data preprocessing"""
    df = df.dropna()
    
    # Column names
    df.columns = df.columns.str.lower().str.replace(' ', '_')
    df['magnitude'] = df['magnitue']
    df = df.drop(['magnitue'], axis=1)

    # Organize label columns
    #     label           2 categories (binary): 0: BenignTraffic, 1: Attack
    #     attack_cat      8 categories: Bening, DDoS, DoS, Recon, Mirai, Web-based, Spoofing, Brute Force
    #     attack_type     34 categories: Benign and 33 attacks
    df['grouped_label'] = df['label'].map(GroupAttacks)
    df.rename(columns={'label': 'attack_type', 'grouped_label': 'attack_cat'}, inplace=True)
    df.replace({'attack_cat': {'BenignTraffic': 'Benign'}, 'attack_type': {'BenignTraffic': 'Benign'}}, inplace=True)
    df['label'] = df['attack_cat'].apply(lambda x: 0 if x == 'Benign' else 1)

    # Convert categorical labels to numerical labels
    label_encoder = LabelEncoder()
    df["attack_cat"] = label_encoder.fit_transform(df["attack_cat"])
    
    unique_attacks = ['Benign'] + sorted(set(df["attack_type"]) - {"Benign"})
    attack_type_mapping = {attack: idx for idx, attack in enumerate(unique_attacks)}
    df["attack_type"] = df["attack_type"].map(attack_type_mapping)

    # Feature Selection
    less_important_features = ['drate', 'fin_flag_number', 'syn_flag_number', 'rst_flag_number', 'ack_flag_number', 'ece_flag_number', 'cwr_flag_number', 'http', 'dns', 'telnet', 'smtp', 'ssh', 'irc', 'tcp', 'udp', 'dhcp', 'arp', 'icmp', 'ipv', 'llc', 'number']
    df = df.drop(columns=less_important_features)

    return df

In [ ]:
df_labels = pd.read_csv("..\\datasets\\df_labels.csv")

In [ ]:
samples = df_labels.groupby('attack_cat_name')['count'].sum().reset_index().sort_values('count', ascending=False)

total = samples['count'].values.sum()
benign = samples[samples['attack_cat_name'] == 'Benign']['count'].values[0]
attacks = (samples[samples['attack_cat_name'] != 'Benign']['count'].values).sum()

print(f"There are {benign} ({benign/total *100:.2f}%) benign samples and {attacks} ({attacks/total*100:.2f}%) attacks.")

In [ ]:
scale = StandardScaler()

def ScaleData_train(df):
    """ Normalize training data features to ensure each has Mean = 0 and Standard Deviation = 1"""
    
    df_scaled = scale.fit_transform(df)
    return pd.DataFrame(df_scaled, columns=df.columns) 

def ScaleData_test(df):
    """ Normalize validation and test data features to ensure each has Mean = 0 and Standard Deviation = 1"""
    
    df_scaled = scale.transform(df)
    return pd.DataFrame(df_scaled, columns=df.columns) 
    
def Split(df, training):
    """Divide DataFrame 'df' in features and labels"""
    
    X = df.drop(columns=['label', 'attack_cat', 'attack_type'])
    X_scaled = ScaleData_train(X) if training else ScaleData_test(X)

    # Discard 'atack_cat' as the label will not be used in any model
    y_label = df['label']
    y_attack = df['attack_type']
    
    return X_scaled, y_label, y_attack


def to_tensor(X, y):
    return TensorDataset(torch.tensor(X.to_numpy(), dtype=torch.float32), torch.tensor(y.to_numpy(), dtype=torch.long))

In [ ]:
nodes = 3 # Number of nodes defined

uniform_partitions = {}
uniform_train_sets = {}
uniform_val_sets = {}

node_data = {}

for i in range(nodes):
    uniform_partitions[i] = preprocessing(pd.read_csv(f"..\\datasets\\UniformPartitions\\uniform_part{i}.csv"))
    
    # Split into Train (70%) and Validation (30%) 
    uniform_train_sets[i], uniform_val_sets[i] = train_test_split(uniform_partitions[i], test_size = 0.3, shuffle=True, random_state=42)
    node_data[i] = Split(uniform_train_sets[i], True), Split(uniform_val_sets[i], False)

Structure of the Dictionary `node_data`:

 * `node_data[i][0]`: returns 3 datasets (`X_train`, `y_train_bin` and `y_train_multi`) of node `i` that will be used for training
 * `node_data[i][1]`: returns 3 datasets (`X_val`, `y_val_bin` and `y_val_multi`) of node `i` that will be used for validation

# **MODELS**

In [ ]:
# import flwr as fl
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix, ConfusionMatrixDisplay, matthews_corrcoef
from collections import defaultdict

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

import optuna
from plotly.io import show
from copy import deepcopy

## **DEEP LEARNING (DL)**

In [ ]:
class DNN(nn.Module):
    def __init__(self, input_dim, hidden_layers, hidden_units, activation, output_dim):
        super(DNN, self).__init__()

        layers = []
        in_features = input_dim

        for _ in range(hidden_layers):
            layers.append(nn.Linear(in_features, hidden_units))
            layers.append(activation())  # Activation function
            in_features = hidden_units

        # Output layer
        layers.append(nn.Linear(in_features, output_dim))
        
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

In [ ]:
class CNN(nn.Module):
    def __init__(self, input_dim, num_filters, fc_units, dropout, num_conv_layers, output_dim):
        super(CNN, self).__init__()
        # self.conv1 = nn.Conv1d(in_channels=1, out_channels=num_filters, kernel_size=3, padding=1)
        self.convs = nn.ModuleList([
            nn.Conv1d(in_channels=1 if i == 0 else num_filters, out_channels=num_filters, kernel_size=3, padding=1)
            for i in range(num_conv_layers)
        ])
        self.fc = nn.Linear(num_filters * input_dim, fc_units)
        self.output = nn.Linear(fc_units, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = x.unsqueeze(1)  # Add channel dimension
        for conv in self.convs:
            x = torch.relu(conv(x))
        # x = torch.relu(self.conv1(x))
        x = x.view(x.size(0), -1)  # Flatten
        x = self.dropout(torch.relu(self.fc(x)))
        x = self.output(x)
        return x

In [ ]:
class CNN_LSTM(nn.Module):
    def __init__(self, input_dim, num_filters, lstm_hidden, num_layers, fc_units, dropout, num_conv_layers, output_dim):
        super(CNN_LSTM, self).__init__()
        # self.conv1 = nn.Conv1d(in_channels=1, out_channels=num_filters, kernel_size=3, padding=1)
        self.convs = nn.ModuleList([
            nn.Conv1d(in_channels=1 if i == 0 else num_filters, out_channels=num_filters, kernel_size=3, padding=1)
            for i in range(num_conv_layers)
        ])
        self.lstm = nn.LSTM(input_size=num_filters, hidden_size=lstm_hidden, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(lstm_hidden, fc_units)
        self.output = nn.Linear(fc_units, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = x.unsqueeze(1)  # Add channel dimension
        for conv in self.convs:
            x = torch.relu(conv(x))
        # x = torch.relu(self.conv1(x))
        x = x.permute(0, 2, 1)  # Reshape for LSTM (batch, seq_len, features)
        x, _ = self.lstm(x)
        x = self.dropout(torch.relu(self.fc(x[:, -1, :])))
        x = self.output(x)
        return x

In [ ]:
def train_pytorch(model, train_loader, val_loader, optimizer, criterion, epochs, threshold=0):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    for epoch in range(epochs):
        model.train()
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()

    # Evaluate on validation set
    model.eval()
    all_preds, all_labels = [], []
    # correct, total = 0, 0
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            outputs = model(X_batch)

            # if threshold != 0 :
            #     probs = F.softmax(outputs, dim=1)  # Convert logits to probabilities
            #     max_probs, predicted = torch.max(probs, dim=1)  # Get highest prob & predicted class
                
            #     # Multiclass classification: assign 'Unknown' class if confidence is below the threshold
            #     predicted[max_probs < threshold] = 0  # Label `0` for 'Unknown'

            # else:
            #     _, predicted = torch.max(outputs, 1)
            _, predicted = torch.max(outputs, 1)
            all_preds.append(predicted.cpu())
            all_labels.append(y_batch.cpu())
    y_pred, y_true = torch.cat(all_preds), torch.cat(all_labels)

    # if threshold != 0:
    #     # Compute accuracy (excluding 'Unknown' cases)
    #     valid_mask = y_pred != 0  # Ignore 'Unknown' class
    #     accuracy = (y_pred[valid_mask] == y_true[valid_mask]).float().mean().item() if valid_mask.any() else 0.0
    #     print(f"ValAcc: {accuracy}")

    # val_acc = correct / total
    Accuracy = accuracy_score(y_true, y_pred)
    print(f"Accuracy: {(Accuracy*100):.2f}%")
    mcc = evaluate(model, [], y_true, y_pred, 'pytorch')
    return model.state_dict(), mcc  # Return local trained model weights and evaluation metrics
    
    

# def test_pytorch(model, X_test, y_test):
#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     model.to(device)
#     model.eval()
#     X_test_tensor = torch.tensor(X_test.to_numpy(), dtype=torch.float32).to(device)
#     y_test_tensor = torch.tensor(y_test.to_numpy(), dtype=torch.long).to(device)

#     with torch.no_grad():
#         outputs = model(X_test_tensor)
#         _, predicted = torch.max(outputs, 1)
#         accuracy = (predicted == y_test_tensor).float().mean().item()

#     return accuracy

## **MACHINE LEARNING (ML)**

In [ ]:
def train_tree_model(model_class, X_train, y_train, X_val, y_val):
    """Train a tree-based model on a given client's data"""
    
    model = model_class
    model.fit(X_train, y_train, eval_set=[(X_val, y_val)])
    return model


def aggregate_tree_models(model_class, models, X_val, y_val, X_train, mode):

    # Simple aggregation: choose the best model based on validation accuracy
    if mode == "Avg":
        best_model = max(models, key=lambda m: m.score(X_val, y_val))
    
    # Train a global XGBoost model using averaged soft predictions from clients
    else:
        soft_predictions = np.mean([model.predict_proba(X_train) for model in models], axis=0)
        pseudo_labels = np.argmax(soft_predictions, axis=1)  # Convert to class labels
    
        # Train new XGBoost model on pseudo-labels
        best_model = model_class
        best_model.fit(X_train, pseudo_labels)
            
    return best_model


def test_tree_model(model, X_test, y_test):
    return model.score(X_test, y_test)

In [ ]:
NUM_CLIENTS = 3  # Number of federated clients
HYPERPARAM_TUNING = True  # Set to False to disable Optuna tuning
EPOCHS = 5  # Training epochs for PyTorch models
input_dim = 25

# **HYPERPARAMETER TUNNING**

## **STUDIES DEFINITION**

In [ ]:
activation_dict = {
    "ReLU": torch.nn.ReLU,
    "LeakyReLU": torch.nn.LeakyReLU,
    "Tanh": torch.nn.Tanh
}

def define_optimizer(model, optimizer_name, lr):
    if optimizer_name == "Adam":
        optimizer = optim.Adam(model.parameters(), lr=lr)
    elif optimizer_name == "SGD":
        optimizer = optim.SGD(model.parameters(), lr=lr)
    else:
        optimizer = optim.RMSprop(model.parameters(), lr=lr) 
    return optimizer


def tune_DNN(trial, X_train, y_train, X_val, y_val, MODE):

    hidden_layers = trial.suggest_int("hidden_layers", 1, 5)
    hidden_units = trial.suggest_int("hidden_units", 16, 128)
    activation_name = trial.suggest_categorical("activation", list(activation_dict.keys()))
    activation_fn = activation_dict[activation_name]
    lr = trial.suggest_float("learning_rate", 1e-4, 1e-1, log=True)
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "SGD", "RMSprop"])
    batch_size = trial.suggest_int('batch_size', 5, 10)

    model = DNN(input_dim, hidden_layers, hidden_units, activation_fns, s, output_dim=CLASSES).to(device)
    criterion = nn.CrossEntropyLoss() if MODE == 'binary' else nn.CrossEntropyLoss(label_smoothing=0.1)
    optimizer = define_optimizer(model, optimizer_name, lr)
    
    train_loader = DataLoader(to_tensor(X_train, y_train), batch_size=2**batch_size, shuffle=False)
    val_loader = DataLoader(to_tensor(X_val, y_val), batch_size=2**batch_size, shuffle=False)
    
    _, mcc = train_pytorch(model, train_loader, val_loader, optimizer, criterion, 10) # Small number of epochs for tuning
    return mcc
    

def tune_CNN(trial, X_train, y_train, X_val, y_val, MODE):
    num_filters = trial.suggest_int("num_filters", 32, 128)
    fc_units = trial.suggest_int("fc_units", 16, 64)
    dropout = trial.suggest_float("dropout", 0.1, 0.5)
    num_conv_layers = trial.suggest_int("num_conv_layers", 1, 3)
    lr = trial.suggest_float("lr", 1e-4, 1e-2, log=True)
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "SGD", "RMSprop"])
    batch_size = trial.suggest_int('batch_size', 5, 10)
    
    model = CNN(input_dim, num_filters=num_filters, fc_units=fc_units, dropout=dropout, num_conv_layers=num_conv_layers, output_dim=CLASSES)
    criterion = nn.CrossEntropyLoss() if MODE == 'binary' else nn.CrossEntropyLoss(label_smoothing=0.1)
    optimizer = define_optimizer(model, optimizer_name, lr)

    train_loader = DataLoader(to_tensor(X_train, y_train), batch_size=2**batch_size, shuffle=False)
    val_loader = DataLoader(to_tensor(X_val, y_val), batch_size=2**batch_size, shuffle=False)
    
    _, mcc = train_pytorch(model, train_loader, val_loader, optimizer, criterion, 10) # Small number of epochs for tuning
    return mcc
    
    
def tune_CNN_LSTM(trial, X_train, y_train, X_val, y_val, MODE):
    num_filters = trial.suggest_int("num_filters", 32, 128)
    lstm_hidden = trial.suggest_int("lstm_hidden", 16, 64)
    num_layers = trial.suggest_int("num_layers", 1, 3)
    fc_units = trial.suggest_int("fc_units", 16, 64)
    dropout = trial.suggest_float("dropout", 0.1, 0.5)
    num_conv_layers = trial.suggest_int("num_conv_layers", 1, 3)
    lr = trial.suggest_float("lr", 1e-4, 1e-2, log=True)
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "SGD", "RMSprop"])
    batch_size = trial.suggest_int('batch_size', 5, 10)
    
    model = CNN_LSTM(input_dim, num_filters=num_filters, lstm_hidden=lstm_hidden, num_layers=num_layers, fc_units=fc_units, dropout=dropout, num_conv_layers=num_conv_layerss, output_dim=CLASSES)
    criterion = nn.CrossEntropyLoss() if MODE == 'binary' else nn.CrossEntropyLoss(label_smoothing=0.1)
    optimizer = define_optimizer(model, optimizer_name, lr)
    
    train_loader = DataLoader(to_tensor(X_train, y_train), batch_size=2**batch_size, shuffle=False)
    val_loader = DataLoader(to_tensor(X_val, y_val), batch_size=2**batch_size, shuffle=False)
    
    _, mcc = train_pytorch(model, train_loader, val_loader, optimizer, criterion, 10) # Small number of epochs for tuning
    return mcc

In [ ]:
def tune_XGB(trial, X_train, y_train, X_val, y_val):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 50, 200),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        'eta': trial.suggest_float('eta', 0.01, 0.3), # learning rate
        'reg_alpha': trial.suggest_int('reg_alpha', 0, 100, step=5), # L1 regularization
        'reg_lambda': trial.suggest_int('reg_lambda', 0, 100, step=5) # L2 regularization
    }
    
    model = XGBClassifier(objective = 'binary:logistic', random_state=42, **params) #  eval_metric = 'auc'
    model.fit(X_train, y_train)
    return evaluate(model, X_val, y_val, [], 'tree')
    

def tune_LGBM(trial, X_train, y_train, X_val, y_val):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 50, 200),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        'reg_alpha': trial.suggest_int('reg_alpha', 0, 100, step=5), # L1 regularization
        'reg_lambda': trial.suggest_int('reg_lambda', 0, 100, step=5) # L2 regularization
    }
    
    model = LGBMClassifier(objective = 'binary', random_state=42, **params)
    model.fit(X_train, y_train)
    return evaluate(model, X_val, y_val, [], 'tree')

In [ ]:
def evaluate(model, X, y_true, y_pred, model_type):
    if model_type =='tree':
        y_pred = model.predict(X)
        
    # accuracy = accuracy_score(y_true, y_pred)
    # precision = precision_score(y_true, y_pred)
    # recall = recall_score(y_true, y_pred)
    # f1 = f1_score(y_true, y_pred)
    mcc = matthews_corrcoef(y_true, y_pred)

    return mcc

In [ ]:
def full_dataset(MODE):
    j = 1 if (MODE == 'binary') else 2
    
    X_train = pd.concat([node_data[i][0][0] for i in range(NUM_CLIENTS)], ignore_index=True)
    y_train = pd.concat([node_data[i][0][j] for i in range(NUM_CLIENTS)], ignore_index=True)
    
    X_val = pd.concat([node_data[i][1][0] for i in range(NUM_CLIENTS)], ignore_index=True)
    y_val = pd.concat([node_data[i][1][j] for i in range(NUM_CLIENTS)], ignore_index=True)

    # Only exclude label 0 ('Benign') if MODE is not 'binary'
    if MODE != 'binary':
        mask_train = y_train != 0
        mask_val = y_val != 0

        X_train, y_train = X_train[mask_train], y_train[mask_train]
        X_val, y_val = X_val[mask_val], y_val[mask_val]

        # Ensure labels are zero-indexed (1-33 -> 0-32)
        y_train = y_train - 1
        y_val = y_val - 1
        
    return X_train, y_train, X_val, y_val

## **ATTACK DETECTION**

In [ ]:
MODE = 'binary'
CLASSES = 2 if MODE == 'binary' else 33 # 33 types of attacks

In [ ]:
best_params, feat_importance, best_scores, studies = {}, {}, {}, {}

def objective(trial, model_name):
    return models_dict[model_name](trial)

X_train, y_train, X_val, y_val = full_dataset(MODE)

models_dict = {
    'DNN': lambda trial: tune_DNN(trial, X_train, y_train, X_val, y_val, MODE),
    'CNN': lambda trial: tune_CNN(trial, X_train, y_train, X_val, y_val, MODE),
    'CNN-LSTM': lambda trial: tune_CNN_LSTM(trial, X_train, y_train, X_val, y_val, MODE),
    'XGB': lambda trial: tune_XGB(trial, X_train, y_train, X_val, y_val),
    'LGBM': lambda trial: tune_LGBM(trial, X_train, y_train, X_val, y_val)
}


for model in models_dict.keys():
    
    print(f"\n\n--------------- {model} ---------------")
    study = optuna.create_study(directions=["maximize"])
    study.optimize(lambda trial: objective(trial, model), n_trials=30)

    best_trial = study.best_trials[0]
    best_params[model] = best_trial.params
    best_scores[model] = best_trial.values
    feat_importance[model] = optuna.importance.get_param_importances(study, target=lambda t: t.values[0])
    studies[model] = study
    
    print(f"    Best parameters:       {best_params[model]}")
    print(f"    Best scores:           {best_scores[model]}")
    print(f"    Parameters importance: {feat_importance[model]}")
    fig = optuna.visualization.plot_param_importances(study)
    show(fig)

### **RESULTS**

In [ ]:
for model, values in best_params.items():
    print(f"\n\n\n ========================== AD: {model} ==========================")
    
    print(f"\nBest hyperparameters: {values}")
    print(f"Best MCC : {best_scores[model]}") 
    print(f"\nHyperparameters importance: {feat_importance[model]}")

## **ATTACK CLASSIFICATION**

In [ ]:
MODE = 'multiclass'
CLASSES = 2 if MODE == 'binary' else 33 # 33 types of attacks

In [ ]:
best_params, feat_importance, best_scores, studies = {}, {}, {}, {}

def objective(trial, model_name):
    return models_dict[model_name](trial)

X_train, y_train, X_val, y_val = full_dataset(MODE)

models_dict = {
    'DNN': lambda trial: tune_DNN(trial, X_train, y_train, X_val, y_val, MODE),
    'CNN': lambda trial: tune_CNN(trial, X_train, y_train, X_val, y_val, MODE),
    'CNN-LSTM': lambda trial: tune_CNN_LSTM(trial, X_train, y_train, X_val, y_val, MODE),
    'XGB': lambda trial: tune_XGB(trial, X_train, y_train, X_val, y_val),
    'LGBM': lambda trial: tune_LGBM(trial, X_train, y_train, X_val, y_val)
}


for model in models_dict.keys():
    
    print(f"\n\n--------------- {model} ---------------")
    study = optuna.create_study(directions=["maximize"])
    study.optimize(lambda trial: objective(trial, model), n_trials=30)

    best_trial = study.best_trials[0]
    best_params[model] = best_trial.params
    best_scores[model] = best_trial.values
    feat_importance[model] = optuna.importance.get_param_importances(study, target=lambda t: t.values[0])
    studies[model] = study
    
    print(f"    Best parameters:       {best_params[model]}")
    print(f"    Best scores:           {best_scores[model]}")
    print(f"    Parameters importance: {feat_importance[model]}")
    fig = optuna.visualization.plot_param_importances(study)
    show(fig)

### **RESULTS**

In [ ]:
for model, values in best_params.items():
    print(f"\n\n\n ========================== AC: {model} ==========================")
    
    print(f"\nBest hyperparameters: {values}")
    print(f"Best MCC : {best_scores[model]}") 
    print(f"\nHyperparameters importance: {feat_importance[model]}")